## Setup

In [1]:
%pip install ipympl
%pip install python-dotenv
%pip install wandb

Note: you may need to restart the kernel to use updated packages.
  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
Using cached python_dotenv-1.1.0-py3-none-any.whl (20 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached wandb-0.19.9-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl.metadata (1.8 kB)
  Using cached sentry_sdk-2.26.1-py2.py3-none-any.whl.metadata (10 kB)
  Using cached setproctitle-1.3.5-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
Using cached wandb-0.19.9-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (20.9 MB)
Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
Using cached sentry_sdk-2.26.1-py2.py3-none-any.whl (340 kB)
Using cached setproctitle-1.3.5-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.wh

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
#import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import random
import wandb

## CNN images model

In [15]:
import torch.nn as nn

class ASLCNN_img(nn.Module):
    def __init__(self):
        super(ASLCNN_img, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=7, stride=2, padding=3), # 400 -> 200
            nn.ReLU(),
            nn.MaxPool2d(2),  # 200 -> 100
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # 100
            nn.ReLU(),
            nn.MaxPool2d(2),  # 100 -> 50
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1), # 50
            nn.ReLU(),
            nn.MaxPool2d(2),  # 50 -> 25
        )
        
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 25 * 25, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 29)  # number of classes
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


In [6]:
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

## Training, validation, test loops

In [7]:
import datetime
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score


# TRAIN FUNCTION
def train(model, train_loader, criterion, optimizer):
    device = torch.device("cuda")
    model.train()
    total_loss = 0
    train_predictions = []
    train_labels = []

    for X_batch, y_batch in train_loader: 
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        train_predictions.append(predicted.cpu())
        train_labels.append(y_batch.cpu())

    epoch_avg_loss = total_loss / len(train_loader)
    return epoch_avg_loss, train_predictions, train_labels


# VALIDATION FUNCTION
def validate(model, val_loader, criterion):
    device = torch.device("cuda")
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    val_predictions = []
    val_labels = []
    
    with torch.no_grad():
        for X_val_batch, y_val_batch in val_loader:
            X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            outputs = model(X_val_batch)
            loss = criterion(outputs, y_val_batch)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == y_val_batch).sum().item()
            total += y_val_batch.size(0)

            val_predictions.append(predicted.cpu())
            val_labels.append(y_val_batch.cpu())

    val_predictions = torch.cat(val_predictions).numpy()
    val_labels = torch.cat(val_labels).numpy()

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * correct / total
    precision = precision_score(val_labels, val_predictions, average='weighted', zero_division=0)
    recall = recall_score(val_labels, val_predictions, average='weighted', zero_division=0)
    conf_matrix = confusion_matrix(val_labels, val_predictions)

    if conf_matrix.shape == (2, 2):
        specificity = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])
    else:
        specificity = 0

    sensitivity = recall
    sk_learn_acc = accuracy_score(val_labels, val_predictions)

    return avg_val_loss, val_accuracy, precision, recall, specificity, sensitivity, sk_learn_acc


# TRAIN + VALIDATE FUNCTION
def train_and_validate(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    device = torch.device("cuda")
    model.to(device)
    current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

    epoch_losses = []
    best_val_accuracy = 0.0
    best_epoch = 0
    best_model_predictions = []
    best_model_labels = []

    for epoch in range(num_epochs):
        train_avg_loss, train_predictions, train_labels = train(model, train_loader, criterion, optimizer)
        epoch_losses.append(train_avg_loss)
        print(f"[Epoch {epoch+1}] Train Loss: {train_avg_loss:.4f}")

        val_avg_loss, val_accuracy, precision, recall, specificity, sensitivity, sk_learn_acc = validate(model, val_loader, criterion)
        print(f"[Epoch {epoch+1}] Val Loss: {val_avg_loss:.4f} | Val Acc: {val_accuracy:.2f}% | Precision: {precision:.2f} | Recall: {recall:.2f}")

        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_epoch = epoch + 1
            best_model_predictions = train_predictions.copy()
            best_model_labels = train_labels.copy()
            torch.save(model.state_dict(), f'best_model_{current_time}.pth')

    print(" Training complete!")
    print(f" Best model: Epoch {best_epoch} with Val Accuracy: {best_val_accuracy:.2f}%")
    
    return model, best_model_predictions, best_model_labels, epoch_losses, best_val_accuracy


In [8]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader

device = torch.device("cuda")

dataset_path_img = '/exchange/dspro2/silent-speech/ASL_Pictures_Dataset'
train_path = f'{dataset_path_img}/Train'
validate_path = f'{dataset_path_img}/Validation'


# Pré-traitement des images
transform = transforms.Compose([
    transforms.Resize((400, 400)),   # Resize les images
    transforms.ToTensor(),           # Convertit en tenseur PyTorch
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalisation (valeurs entre -1 et 1)
])

# Chargement des datasets avec ImageFolder
train_dataset = ImageFolder(root=train_path, transform=transform)
val_dataset = ImageFolder(root=validate_path, transform=transform)

# Création des DataLoaders
batch_size = 256

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=8)


In [16]:
# CNN IMAGES

num_classes = 29
device = torch.device("cuda")
model = ASLCNN_img().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.0001) #weight_decay = regularization
criterion = nn.CrossEntropyLoss()

# Call the train_and_validate function to train and validate the model
trained_model, best_model_predictions, best_model_labels, epoch_losses, sk_learn_acc = train_and_validate(
    model, train_loader, val_loader, criterion, optimizer, num_epochs=2)


[Epoch 1] Train Loss: 0.8157
[Epoch 1] Val Loss: 0.0864 | Val Acc: 97.56% | Precision: 0.98 | Recall: 0.98
[Epoch 2] Train Loss: 0.1144
[Epoch 2] Val Loss: 0.0188 | Val Acc: 99.59% | Precision: 1.00 | Recall: 1.00
 Training complete!
 Best model: Epoch 2 with Val Accuracy: 99.59%


In [ ]:
# Get a single batch from the DataLoader
sample_batch = next(iter(train_loader))

# Unpack the batch
print(f"Type of sample_batch: {type(sample_batch)}")
print(f"Length of sample_batch: {len(sample_batch)}")

# Print what's inside
for i, item in enumerate(sample_batch):
    print(f"Item {i}: type = {type(item)}, shape = {item.shape if hasattr(item, 'shape') else 'N/A'}")
